# Water Column Modeling
> This notebook will perform our water column modeling for each entry of the vegetation density data, giving us flow properties (U, Q2, Kz etc..) for each vegetation density profile

Developed by Alexandre Erich Sebastien Georges, PhD Student in Environmental Engineering at UC Berkeley <br> Fall 2022

In [2]:
# Import water column model class and components
from water_column_model.column import Column
from water_column_model.advance import *
from water_column_model.params import *
from water_column_model.params import A,B,C,E
# Data Science and Visualization Imports
import plotly.graph_objects as go
import plotly.express as px
import pandas as pd
import xarray as xr
import numpy as np
# Miscellaneous
from tqdm import tqdm

## Importing USGS Data

In [3]:
filename = 'data/USGS_veg_density_09.16.2022.csv'
df = pd.read_csv(filename)
df.head()

,Season,date,site,plot_ID,Elevation,channel_dist,ave_ht,cover,vol_est,vol_calc,density_naive,density_final,frontal_total,frontal_5,frontal_10,frontal_15,frontal_20,frontal_25,frontal_30
0,Summer,7/28/2021,Bay,A0.5,2.1313,0.5,6.5,0.10,6500,233.351057,267.040006,630.481094,1103.449997,848.807690,254.642307,0.000000,0.000000,0.0,0.0
1,Summer,7/28/2021,Bay,A2,2.3217,2.0,11.0,0.45,49500,1777.058049,2033.612355,2837.164924,8403.196133,3819.634606,3819.634606,763.926921,0.000000,0.0,0.0
2,Summer,7/28/2021,Bay,A6,2.2421,6.0,19.0,0.55,104500,3751.566993,4293.181638,3467.646018,17740.080726,4668.442296,4668.442296,4668.442296,3734.753837,0.0,0.0
3,Summer,7/28/2021,Bay,A12,2.2417,12.0,15.0,0.75,112500,4038.768294,4621.846261,4728.608206,19098.173030,6366.057677,6366.057677,6366.057677,0.000000,0.0,0.0
4,Summer,7/28/2021,Bay,A24,2.2217,24.0,11.0,0.55,60500,2171.959838,2485.526211,3467.646018,10270.573052,4668.442296,4668.442296,933.688459,0.000000,0.0,0.0


Temporary: Only using data from Bay Sites

In [4]:
df['ave_ht'].describe()

count    114.000000
mean      15.346491
std        5.261787
min        4.000000
25%       12.000000
50%       15.000000
75%       19.000000
max       30.000000
Name: ave_ht, dtype: float64

In [5]:
#df = df[df['site'] == 'Bay']
#df['site'].value_counts()

## Which type of run is this? (Baseline [0] or Vegetated [1])

In [6]:
runtype_input = int(input("Which type of run is this? (Baseline [0] or Vegetated [1]) "))
if runtype_input == 1:
    density_alpha = float(input('Select a value [0-1] for alpha density coefficient: '))
else:
    density_alpha = 0
print("Runtype is :" + str(runtype_input))

Runtype is :0


## Making Water Columns for Data Entries

In [7]:
# Setting max water column height (arbitrary for now, but based on maximum vegetation height from data)
H = 2.8072 # centimeters
# Adjusting Tide level to topography (elevation) for correct water column height for each site
df['wc_height'] = H - df['Elevation']

In [8]:
# for a given set of parameteres, create a column, run the model and return Flow Properties
def run_column_model(N, H, L, SMALL, params, veg_density, veg_ht, density_alpha):
    # Initializing Column Object
    col = Column(N, H, L, SMALL)
    A = params[0]
    B = params[1]
    C = params[2]
    Sq = params[3]
    kappa = params[4]
    nu = params[5]
    g = params[6]
    rho0 = params[7]
    alpha = params[8]
    # Setting up vlaues and importing Vegetation Distribution from Data
    col = col.setup(A, B, C, Sq, kappa, SMALL, nu, g, rho0, alpha)
    col = col.import_veg(density_alpha, veg_density, veg_ht)
    # Running Model for Col.M amount of time (See Column Class to change M)
    t = [] 
    for i in tqdm(range(col.M), leave=False):
        t.append(col.dt*(i))
        # Unew, Cnew, Qnew, Q2new, Q2Lnew, rhonew, Lnew, nu_tnew, Kznew, Kqnew, N_BVnew, N_BVsqnew
        [col.U, col.scalar, col.Q, col.Q2, col.Q2L, col.rho, col.L, col.nu_t, col.Kz, col.Kq, col.N_BV, col.N_BVsq] = wc_advance(col, t_px, px0, t[i])
        # Have TQDM instead print('Step! t=' +str(t[i])+'s')
    ### Only for now ### Return Velocity Profile ### Will implement returning more later
    col_res = pd.DataFrame({'U Velocity':col.U,
                            'Q':col.Q,
                            'Q2':col.Q2,
                            'Q2L':col.Q2L,
                            'Z':col.z,
                            'Kq':col.Kq,
                            'nu_t':col.nu_t,
                            })
    return col_res

In [11]:
# New dataframe without unneccessary columns
wcolumns = df.drop(['plot_ID', 'frontal_5', 'frontal_10', 'frontal_15', 'frontal_20', 'frontal_20', 'frontal_25', 'frontal_30'], axis=1)
# Pulling parameters
params = [A, B, C, Sq, kappa, nu, g, rho0, alpha]

# Initializing and running water column model for each entry
col_res = wcolumns.apply(lambda x: run_column_model(80,(x.wc_height*100),0,SMALL,params,x.density_final,x.ave_ht,density_alpha), axis=1)

Getting locations

In [25]:
# Reading Dataset with locations
loc_data = 'data/EdenLanding_RTK_Data.csv'
locs = pd.read_csv(loc_data)
# Keeping LatLon columns and identifying columns
locs = locs[['Easting', 'Northing','Transect','Distance']]
# Transforming identifying columns to match plot_ID from USGS data
locs = locs.drop(locs[locs['Distance']==0].index)
locs = locs.sort_values(['Transect','Distance'], ascending=True)
locs['Distance'] = locs['Distance'].apply(lambda x: 72 if x == 75 else x)
locs['Distance'] = locs['Distance'].apply(lambda x: str(int(x)) if x >= 1 else str(x))
# Creating plot_ID column
locs['plot_ID'] = locs['Transect'] + (locs['Distance']).astype(str)
# Dropping duplicate IDs (locations from similar IDs are very close to each other anyway)
locs = locs.drop_duplicates('plot_ID')
locs.head(20)

,Easting,Northing,Transect,Distance,plot_ID
23,575481.457229,4.159935e+06,A,0.5,A0.5
14,575483.018742,4.159936e+06,A,2,A2
17,575486.683492,4.159938e+06,A,6,A6
8,575492.070254,4.159941e+06,A,12,A12
9,575502.472088,4.159947e+06,A,24,A24
0,575523.476358,4.159958e+06,A,48,A48
115,575544.438181,4.159970e+06,A,72,A72
52,575509.572719,4.159904e+06,B,6,B6
34,575514.765340,4.159907e+06,B,12,B12
110,575525.241876,4.159913e+06,B,24,B24


Turning results into a Xarray for easy processing and visualization

In [26]:
#xr.Dataset(col_res.tolist())
wc_ds = xr.concat([df.to_xarray() for df in col_res], dim="Model Runs")
wc_ds = wc_ds.assign_attrs(description='Model run results for all sites.')

## Export Results Data

In [27]:
if runtype_input == 1:
    export_wc_path = 'results_data/wc_runs_all.nc'
    wc_ds.to_netcdf(path=export_wc_path, mode='w')
elif runtype_input == 0:
    export_baseline = 'results_data/baseline_wc_full.nc'
    wc_ds.to_netcdf(path=export_baseline, mode='w')
else:
    print('Wrong value for runtype error, please rerun (and modify this to throw error at beginning')

export_input_path = 'results_data/usgs_data_full.csv'
export_locs_path = 'results_data/locs_data.csv'
df.to_csv(export_input_path)
locs.to_csv(export_locs_path)